## 2 Data Pre-processing

下面的文字部分默认不放qmd里，代码前面有 #not in QMD 的不放QMD，是不影响数据清理、用于测试和提供分析数据的代码块

**step:**

load (remote) data(download/upload to github/create url/load into project): url-load code

select the needed columns

deal with Nan and clean(text/numeric(float/int))

basic description of data

_main scources:practical 6_

### Loading the Dataset

In [1]:
#library packages
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
from requests import get
from functools import wraps

数据来源：Inside Airbnb，2025年9月15日数据

In [2]:
# read data from github URL
ymd  = "20250914"
city = "London"
host = "https://raw.githubusercontent.com/mixedmint/fsds_group/data_preprocessing/data/raw"
url  = f'{host}/{ymd}-{city}-listings.csv.gz'

# # read data from github URL
# ymd  = "20250615"
# city = "London"
# host = "https://raw.githubusercontent.com/mixedmint/fsds_group/data_preprocessing/data/raw"
# url  = f'{host}/{ymd}-{city}-listings.csv.gz'

df = pd.read_csv(url, compression="gzip", low_memory=False)

In [3]:
#not in QMD
pd.set_option('display.float_format', lambda x: '%.2f' % x)
#to view the full version of the charts below
pd.set_option('display.max_rows', 200) 
pd.set_option('display.max_columns', 200)
#to reset the display option
#pd.reset_option('display.max_rows')

### Selecting Data

原始数据结构：

In [4]:
#not in QMD
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 96,871 x 79


In [5]:
# not in QMD
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96871 entries, 0 to 96870
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            96871 non-null  int64  
 1   listing_url                                   96871 non-null  object 
 2   scrape_id                                     96871 non-null  int64  
 3   last_scraped                                  96871 non-null  object 
 4   source                                        96871 non-null  object 
 5   name                                          96871 non-null  object 
 6   description                                   94421 non-null  object 
 7   neighborhood_overview                         41208 non-null  object 
 8   picture_url                                   96865 non-null  object 
 9   host_id                                       96871 non-null 

选择研究可能用到的列：
**这里有关于数据的定义，来源Inside Airbnb的data dictionary**

In [6]:
#select the needed columns
cols = ['id', 
        'listing_url', 
        'last_scraped', #UTC. The date and time this listing was "scraped"
        'name', 
        'host_id', 
        'host_name', 
        'host_since', #The date the host/user was created
        'host_location', 
        'host_is_superhost',
        'host_listings_count', #The number of listings the host has (per Airbnb unknown calculations)
        'latitude', #WGS84
        'longitude', 
        'property_type', #Hotels and Bed and Breakfasts are described as such by their hosts in this field
        'room_type', 
        'accommodates', #The maximum capacity of the listing
        'bathrooms', 
        'bedrooms', 
        'beds',
        'price',#daily price in local currency. 
        'minimum_nights', #minimum number of night stay for the listing (calendar rules may be different)
        'maximum_nights', 
        'availability_365', #avaliability_x. The availability of the listing x days in the future as determined by the calendar. 
        #Note a listing may not be available because it has been booked by a guest or blocked by the host.
        'number_of_reviews',#he number of reviews the listing has
        'number_of_reviews_ltm',#he number of reviews the listing has (in the last 12 months)
        'review_scores_rating',
        'reviews_per_month']

In [7]:
#not in QMD
print(f"Cols contains {len(cols)} columns.")

Cols contains 26 columns.


将原始数据变为筛选了列的数据：

In [8]:
df = df[cols]

In [9]:
#not in QMD
df.sample(3, random_state=42)

,id,listing_url,last_scraped,name,host_id,host_name,host_since,host_location,host_is_superhost,host_listings_count,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,price,minimum_nights,maximum_nights,availability_365,number_of_reviews,number_of_reviews_ltm,review_scores_rating,reviews_per_month
56789,1001555573812356101,https://www.airbnb.com/rooms/1001555573812356101,2025-09-14,Luxury Modern Garden Studio |Gym Access| Studio S,520027398,Taiwo,2023-06-14,"London Borough of Barking and Dagenham, United...",f,3.00,51.54,0.12,Entire place,Entire home/apt,2,1.00,1.00,1.00,$78.00,1,365,355,101,46,4.57,4.42
4670,7160564,https://www.airbnb.com/rooms/7160564,2025-09-17,Sunny family house and large garden,9314436,Alice,2013-10-08,"London, United Kingdom",f,1.00,51.58,-0.11,Entire home,Entire home/apt,8,NaN,5.00,NaN,NaN,1,1125,0,0,0,NaN,NaN
29262,38201478,https://www.airbnb.com/rooms/38201478,2025-09-16,Super central Holborn London Luxurious 2beds flat,7757402,Chen,2013-07-27,"London, United Kingdom",f,2.00,51.52,-0.11,Entire condo,Entire home/apt,5,2.50,2.00,2.00,$306.00,5,20,16,9,4,4.89,0.23


In [10]:
#not in QMD
#show the counts of null value of all columns
df.isnull().sum(axis=0).sort_values(ascending=False)[:12]

beds                    34920
price                   34908
bathrooms               34846
reviews_per_month       24122
review_scores_rating    24122
host_location           23771
bedrooms                12775
host_is_superhost        1766
host_name                  43
host_since                 41
host_listings_count        41
name                        0
dtype: int64

In [11]:
#delete the rows with more than 5 null columns
probs = df.isnull().sum(axis=1)
cutoff = 5
df.drop(probs[probs > cutoff].index, inplace=True)
print(f"df contains {df.shape[0]:,} rows.")

df contains 90,674 rows.


检查property_type和room_type的类别情况：
需要注意，我们说的entire是指room_type的Entire home/apt，而property_type主要用于分析后面airbnb房源聚集区域与property类别？（比如筛选出整栋别墅等，这类房源的房东如果从airbnb中获得大量收入，其实是不利于airbnb本身的初衷和社会住房公平的）

**(notes from the meta data dictionary)the definition of different room_type:**
[Entire home/apt|Private room|Shared room|Hotel]

All homes are grouped into the following three room types:

Entire place
Entire places are best if you're seeking a home away from home. With an entire place, you'll have the whole space to yourself. This usually includes a bedroom, a bathroom, a kitchen, and a separate, dedicated entrance. Hosts should note in the description if they'll be on the property or not (ex: "Host occupies first floor of the home"), and provide further details on the listing.

Private rooms
Private rooms are great for when you prefer a little privacy, and still value a local connection. When you book a private room, you'll have your own private room for sleeping and may share some spaces with others. You might need to walk through indoor spaces that another host or guest may occupy to get to your room.

Shared rooms
Shared rooms are for when you don't mind sharing a space with others. When you book a shared room, you'll be sleeping in a space that is shared with others and share the entire space with other people. Shared rooms are popular among flexible travelers looking for new friends and budget-friendly stays.

In [12]:
#not in QMD
#check the categories inside 'property_type','room_type'
cats = ['property_type','room_type']
#the categories in property_type:
df[cats[0]].value_counts()

property_type
Entire rental unit                    40146
Private room in rental unit           11831
Private room in home                  10193
Entire home                            8899
Entire condo                           8171
Private room in condo                  3028
Entire serviced apartment              1843
Private room in townhouse              1104
Entire townhouse                       1046
Room in hotel                           946
Private room in bed and breakfast       456
Private room in guesthouse              346
Entire loft                             332
Entire guesthouse                       216
Room in boutique hotel                  199
Entire guest suite                      170
Private room in guest suite             165
Private room in loft                    143
Private room in serviced apartment      125
Room in aparthotel                       89
Entire villa                             79
Private room                             70
Boat              

In [13]:
#not in QMD
#the categories in room_type:
df[cats[1]].value_counts()

room_type
Entire home/apt    61455
Private room       28903
Shared room          208
Hotel room           108
Name: count, dtype: int64

### Fixing Data Types

Category

In [14]:
cats = ['property_type','room_type']
for c in cats:
    df[c] = df[c].astype('category')

In [15]:
#not in QMD
df.sample(5, random_state=43)[cats]

,property_type,room_type
79337,Entire rental unit,Entire home/apt
89854,Room in hotel,Private room
19781,Entire rental unit,Entire home/apt
50452,Entire rental unit,Entire home/apt
30795,Entire rental unit,Entire home/apt


In [16]:
#not in QMD
print(f"Now {cats[1]} is of type '{df[cats[1]].dtype}'", "\n")

Now room_type is of type 'category' 



Bool

In [17]:
bools = ['host_is_superhost']
for b in bools:
    print(f"Converting {b}")
    df[b] = df[b].replace({'f':False, 't':True}).astype('bool')

Converting host_is_superhost


In [18]:
#not in QMD
df.sample(5, random_state=43)[bools]

,host_is_superhost
79337,False
89854,False
19781,False
50452,False
30795,False


In [19]:
#not in QMD
print(f"Currently {bools[0]} is of type '{df[bools[0]].dtype}'", "\n")

Currently host_is_superhost is of type 'bool' 



Date

In [20]:
dates = ['last_scraped','host_since']
for d in dates:
    print("Converting " + d)
    df[d] = pd.to_datetime(df[d])

Converting last_scraped
Converting host_since


In [21]:
#not in QMD
df.sample(5, random_state=43)[dates]

,last_scraped,host_since
79337,2025-09-15,2016-07-07
89854,2025-09-16,2018-02-23
19781,2025-09-15,2012-05-02
50452,2025-09-16,2015-02-16
30795,2025-09-17,2017-07-18


In [22]:
#not in QMD
print(f"Currently {dates[1]} is of type '{df[dates[1]].dtype}'", "\n")

Currently host_since is of type 'datetime64[ns]' 



Int

In [23]:
ints  = ['id','host_id','host_listings_count','accommodates',
         'beds','minimum_nights','maximum_nights','availability_365','number_of_reviews',
        'number_of_reviews_ltm']
for i in ints:
    print(f"Converting {i}")
    try:
        df[i] = df[i].astype('float').astype('int')
    except ValueError as e:
        print("  - !!!Converting to unsigned 16-bit integer!!!")
        df[i] = df[i].astype('float').astype(pd.UInt16Dtype())

Converting id
Converting host_id
Converting host_listings_count
  - !!!Converting to unsigned 16-bit integer!!!
Converting accommodates
Converting beds
  - !!!Converting to unsigned 16-bit integer!!!
Converting minimum_nights
Converting maximum_nights
Converting availability_365
Converting number_of_reviews
Converting number_of_reviews_ltm


Str

In [24]:
money = ['price']
for m in money:
    df[m] = df[m].str.replace('$','', regex=False).str.replace(',','', regex=False).astype('float')

In [25]:
#not in QMD
df.sample(5, random_state=43)[money]

,price
79337,188.00
89854,96.00
19781,119.00
50452,397.00
30795,NaN


In [26]:
#not in QMD
print(f"Currently {money[0]} is of type '{df[money[0]].dtype}'", "\n")

Currently price is of type 'float64' 



In [27]:
#not in QMD
#validation
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90674 entries, 0 to 96870
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     90674 non-null  int64         
 1   listing_url            90674 non-null  object        
 2   last_scraped           90674 non-null  datetime64[ns]
 3   name                   90674 non-null  object        
 4   host_id                90674 non-null  int64         
 5   host_name              90661 non-null  object        
 6   host_since             90663 non-null  datetime64[ns]
 7   host_location          69534 non-null  object        
 8   host_is_superhost      90674 non-null  bool          
 9   host_listings_count    90663 non-null  UInt16        
 10  latitude               90674 non-null  float64       
 11  longitude              90674 non-null  float64       
 12  property_type          90674 non-null  category      
 13  room_t

### Cleaning Outliers 

data cleaning and basic description(used for analysis)

**关于营业额计算，来自gpt：**
可以根据 reviews 推算 occupancy → 估算 revenue，这正是全世界 Airbnb 研究论文都在做的。

著名论文：
Wachsmuth & Weisler (2018) "The High Cost of Short-Term Rentals in New York City"
Barron, Kung & Proserpio (2021) “Airbnb and Housing Rents”
Inside Airbnb 官方 methodology

核心假设（全世界论文通用）：
大约 50–70% 的 Airbnb 住客会留下 review。所以：
reviews_per_month / 0.5 ≈ bookings per month
bookings * price ≈ revenue

也就是说：如果一个 listing 每月有 2 条评论，假设每 2 个住客有 1 个给评论（50% review rate）→ 真实入住次数大约是 4 次
再结合：
price（每晚价格）
minimum_nights（最低入住天数）
平均入住天数 = 文献一般假设 3–5 nights
就可以估算营业额。


房东实际收入=营业额-(每次的)清洁费*次数

price:

In [28]:
#not in QMD
df['price'].map('{:.2f}'.format)
df['price'].describe()

count     61949.00
mean        229.87
std        4438.05
min           7.00
25%          77.00
50%         135.00
75%         221.00
max     1085147.00
Name: price, dtype: float64

In [29]:
df = df[df['price'] <= 3000]

In [30]:
#not in QMD
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 61,781 x 26


In [31]:
#not in QMD
df['price'].describe()

count   61781.00
mean      185.67
std       198.19
min         7.00
25%        77.00
50%       135.00
75%       220.00
max      3000.00
Name: price, dtype: float64

host_listings_count:

In [32]:
#not in QMD
df['host_listings_count'].describe()

count   61770.00
mean       68.06
std       481.01
min         1.00
25%         1.00
50%         3.00
75%        17.00
max      5469.00
Name: host_listings_count, dtype: Float64

In [33]:
df = df[df['host_listings_count'] <= 50]

In [34]:
#not in QMD
df['host_listings_count'].describe()

count   54447.00
mean        7.59
std        10.70
min         1.00
25%         1.00
50%         2.00
75%         9.00
max        50.00
Name: host_listings_count, dtype: Float64

minimum_nights:

In [35]:
#not in QMD
df['minimum_nights'].describe()

count   54447.00
mean        4.73
std        16.22
min         1.00
25%         1.00
50%         2.00
75%         3.00
max       999.00
Name: minimum_nights, dtype: float64

In [36]:
df = df[df['minimum_nights'] < 999]

In [37]:
#not in QMD
df['minimum_nights'].describe()

count   54446.00
mean        4.72
std        15.65
min         1.00
25%         1.00
50%         2.00
75%         3.00
max       730.00
Name: minimum_nights, dtype: float64

maximum_nights:

In [38]:
#not in QMD
df['maximum_nights'].describe()

count       54446.00
mean        10034.29
std       2249347.14
min             1.00
25%            90.00
50%           365.00
75%           365.00
max     524855552.00
Name: maximum_nights, dtype: float64

availability_365:

In [39]:
#not in QMD
df['availability_365'].describe()

count   54446.00
mean      213.06
std       121.61
min         0.00
25%        90.00
50%       242.00
75%       331.00
max       365.00
Name: availability_365, dtype: float64

number_of_reviews:

In [40]:
#not in QMD
df['number_of_reviews'].describe()

count   54446.00
mean       28.86
std        61.07
min         0.00
25%         1.00
50%         7.00
75%        29.00
max      1902.00
Name: number_of_reviews, dtype: float64

number_of_reviews_ltm:

In [42]:
#not in QMD
df['number_of_reviews_ltm'].describe()

count   54446.00
mean        8.30
std        14.14
min         0.00
25%         0.00
50%         3.00
75%        10.00
max       390.00
Name: number_of_reviews_ltm, dtype: float64

review_scores_rating:

In [43]:
#not in QMD
df['review_scores_rating'].describe()

count   42963.00
mean        4.69
std         0.49
min         1.00
25%         4.61
50%         4.83
75%         5.00
max         5.00
Name: review_scores_rating, dtype: float64

reviews_per_month:

In [44]:
#not in QMD
df['reviews_per_month'].describe()

count   42963.00
mean        1.29
std         1.41
min         0.01
25%         0.34
50%         0.83
75%         1.74
max        36.96
Name: reviews_per_month, dtype: float64

reviews_per_month(calculated by inside airbnb)：
The average number of reviews per month the listing has over the lifetime of the listing.

Psuedocoe/~SQL:

IF scrape_date - first_review <= 30 THEN number_of_reviews
ELSE number_of_reviews / ((scrape_date - first_review + 1) / (365/12))

In [45]:
#not in QMD
print(f"Data frame is {df.shape[0]:,} x {df.shape[1]}")

Data frame is 54,446 x 26


## Overview of the London Airbnb Market

这个部分我觉得应该放到out of control的，所以在这里我只写一下思路：

1 airbnb listing规模：
（1）airbnb房源数量=数据的行数（我们的数据是2025年9月的）
（2）london所有房源数量（我把gpt提供的方法写在共享文档2.2的地方了，找数据的时候最好能找到接近2025年的，如果没有，要说明数据局限性（论文里的2018数据：Based on the Valuation Office Agency data, there are approximately 3.4 million dwellings in London,）

2.airbnb的空间分布图
（1）listings点分布图
（2）按某个行政区范围（比如LA?）汇总listings点数量，绘制以listings的sum为数值的分布图，以及各个room type的占比


*关于论文里misuse的房子的定义（请参考原文）：应该包括三个要求，listings是entire的(room_type == Entire home/apt)且availablity_365大于180天且房东有多套房源
the proportion of **entire Airbnb properties** according to the **annual availability (if listings are available for more than 180 days)**, as well as whether the hosts have **multiple listings** – indicating possible misuse.